# Capstone Project  - Final Assignment (Week 2)
## Battle of Neighborhoods

#### 1. Import Library and Modules

In [9]:
#!conda install -c conda-forge folium=0.5.0 --yes
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
from re import search
from pandas import DataFrame
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import folium

#### 2. Read Survery Data and Neighborhood Dataset

In [10]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client_8dbf0725bdee4042881dc53e79d63a02 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zTqHmG8aDPPhmzpHscXqnfGkPPBF4uv49zpq57vQ0QKp',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_8dbf0725bdee4042881dc53e79d63a02.get_object(Bucket='capstoneproject-donotdelete-pr-enuemg2yqmnfxz',Key='AuburnHills_Neighborhoods.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_neighborhood = pd.read_csv(body)
print('***** List of Neighborhoods ******')
print (df_neighborhood.head(10))

body2 = client_8dbf0725bdee4042881dc53e79d63a02.get_object(Bucket='capstoneproject-donotdelete-pr-enuemg2yqmnfxz',Key='SurveyData.csv')['Body']
if not hasattr(body2, "__iter__"): body2.__iter__ = types.MethodType( __iter__, body2 )

df_unique = pd.read_csv(body2)
print('***** Survey Data of Most Visited Places by the Customer')
print(df_unique.head(15))


***** List of Neighborhoods ******
   Zip Code     Neighborhood State   Latitude  Longitude
0     48070  Hutington Woods    MI  42.480810 -83.166314
1     48067     Royal Oak L1    MI  42.489838 -83.145607
2     48034    Southfield L1    MI  42.497360 -83.291173
3     48076    Southfield L2    MI  42.498356 -83.232383
4     48072          Berkley    MI  42.499015 -83.185144
5     48068     Royal Oak L2    MI  42.489296 -83.145537
6     48334       Farmington    MI  42.498530 -83.359676
7     48071  Madison Heights    MI  42.485713 -83.100312
8     48073     Royal Oak L3    MI  42.520079 -83.164897
9     48025         Franklin    MI  42.521489 -83.305985
***** Survey Data of Most Visited Places by the Customer
               User Category
0                Supermarket
1              Grocery Store
2                       Fuel
3               Noodle House
4         Mexican Restaurant
5                   Pharmacy
6        Arts & Crafts Store
7              Big Box Store
8   Mediterranean Re

#### 3. Query Foursquare API to get all the outlets/venues across neighborhoods

In [11]:
CLIENT_ID = 'AUKGWAYNL30SXDJ5T01V12FPPATOQN2IBCZFJS2XFKVPCJ1N'
CLIENT_SECRET = 'BMW3CTPXW2V3UZZVYIBE22QVIIJMHOKLL3GF5LLGYVZFVK02'
VERSION = '20180604'
LIMIT = 100
radius = 5632.7

fsq_df = []
fsq_cat_df = []
fsq_neigh_df = []

df_fsq = pd.DataFrame()
df_fsq_cat = pd.DataFrame()
df_final = pd.DataFrame()
null = pd.Series(['nan'])

for name,lat,lng in zip(df_neighborhood['Neighborhood'], df_neighborhood['Latitude'], df_neighborhood['Longitude']):
    for cat in df_unique['User Category']:
        search_query = cat
        latitude = lat
        longitude = lng
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
        venues = results['response']['venues']
        dataframe = json_normalize(venues)
    
        if dataframe.empty==False:
            filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
            dataframe_filtered = dataframe.loc[:, filtered_columns]
        
#   function that extracts the category of the venue

            def get_category_type(row):
                try:
                    categories_list = row['categories']
                except:
                        categories_list = row['venue.categories']
        
                if len(categories_list) == 0:
                    return None
                else:
                    return categories_list[0]['name']

# filter the category for each row
            dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
            dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
            dataframe_filtered = dataframe_filtered[dataframe_filtered['categories']==cat]
            fsq_df.append(dataframe_filtered['name'])
            fsq_cat_df.append(dataframe_filtered['categories'])              
    
        else:
            fsq_df.append(null)
            fsq_cat_df.append(null)            
        
    df_fsq = pd.concat(fsq_df, axis=0)
    df_fsq = pd.DataFrame(df_fsq, columns = ['Outlets'])
    df_fsq.reset_index(drop=True, inplace=True)

    df_fsq_cat = pd.concat(fsq_cat_df, axis=0)
    df_fsq_cat = pd.DataFrame(df_fsq_cat, columns = ['Categories'])
    df_fsq_cat.reset_index(drop=True, inplace=True)

    for fill in df_fsq_cat['Categories']:
        df_fsq_cat['Neighborhood'] = name


    df_merged = pd.concat([df_fsq, df_fsq_cat], axis = 1) 
    df_final = df_final.append(df_merged)
    df_sorted = df_final
    
df_merged = []
df_fsq = []
df_fsq_cat =[]

df_sorted.head(50)

,Outlets,Categories,Neighborhood
0,Kroger Grocery Pickup and Delivery,Grocery Store,Hutington Woods
1,Gordon Food Service Store,Grocery Store,Hutington Woods
2,Noodles & Company,Noodle House,Hutington Woods
3,Noodle Topia,Noodle House,Hutington Woods
4,Margaritas Mexican Restaurant,Mexican Restaurant,Hutington Woods
5,Camelia's Mexican Grill,Mexican Restaurant,Hutington Woods
6,Margarita's Authentic Mexican Cuisine,Mexican Restaurant,Hutington Woods
7,Qdoba Mexican Grill,Mexican Restaurant,Hutington Woods
8,Qdoba Mexican Grill,Mexican Restaurant,Hutington Woods
9,CVS pharmacy,Pharmacy,Hutington Woods


#### 4. One hot Encoding and Grouping Dataframe

In [12]:
df = df_sorted

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df_mod= df.drop_duplicates(subset=['Outlets','Neighborhood'], keep = 'first')
df_mod.reset_index(inplace=True, drop=True)

df_onehot = pd.get_dummies(df_mod[['Categories']], prefix="", prefix_sep="")
df_onehot['Neighborhood'] = df_mod['Neighborhood']
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]
df_onehot.head(10)

,Neighborhood,Arts & Crafts Store,Convenience Store,Grocery Store,Hardware Store,Mediterranean Restaurant,Mexican Restaurant,Noodle House,Pharmacy,Pizza Place,Shipping Store,Supermarket,Taco Place,nan
0,Hutington Woods,0,0,1,0,0,0,0,0,0,0,0,0,0
1,Hutington Woods,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Hutington Woods,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Hutington Woods,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Hutington Woods,0,0,0,0,0,1,0,0,0,0,0,0,0
5,Hutington Woods,0,0,0,0,0,1,0,0,0,0,0,0,0
6,Hutington Woods,0,0,0,0,0,1,0,0,0,0,0,0,0
7,Hutington Woods,0,0,0,0,0,1,0,0,0,0,0,0,0
8,Hutington Woods,0,0,0,0,0,0,0,1,0,0,0,0,0
9,Hutington Woods,0,0,0,0,0,0,0,1,0,0,0,0,0


#### 5. Applying k-means Cluster Algorithm

In [13]:
df_grouped = df_onehot.groupby('Neighborhood').sum().reset_index()
X = df_grouped.values[:,1:]
X = np.nan_to_num(X)
cluster_group = StandardScaler().fit_transform(X)

k_cluster = 5
cluster_group = df_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters=k_cluster, random_state=0).fit(cluster_group)
df_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

df_cluster = pd.merge(df_grouped, df_neighborhood, on='Neighborhood')
df_cluster.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,Cluster Labels,Neighborhood,Arts & Crafts Store,Convenience Store,Grocery Store,Hardware Store,Mediterranean Restaurant,Mexican Restaurant,Noodle House,Pharmacy,Pizza Place,Shipping Store,Supermarket,Taco Place,nan,Zip Code,State,Latitude,Longitude
0,1,Auburn Hills L1,8.0,40.0,12.0,40.0,27.0,39.0,5.0,102.0,226.0,5.0,1.0,8.0,1.0,48321,MI,42.638200,-83.295400
1,1,Auburn Hills L2,8.0,40.0,12.0,42.0,27.0,39.0,5.0,106.0,233.0,5.0,1.0,8.0,1.0,48326,MI,42.678552,-83.235753
2,4,Berkley,4.0,10.0,3.0,14.0,10.0,10.0,3.0,49.0,55.0,2.0,0.0,4.0,1.0,48072,MI,42.499015,-83.185144
3,0,Birmingham L1,6.0,13.0,7.0,23.0,17.0,20.0,5.0,72.0,101.0,3.0,0.0,4.0,1.0,48012,MI,42.544339,-83.203559
4,0,Birmingham L2,6.0,13.0,7.0,23.0,18.0,20.0,5.0,73.0,104.0,3.0,0.0,4.0,1.0,48009,MI,42.543498,-83.203264
5,0,Bloomfield Hills L1,6.0,13.0,7.0,23.0,18.0,20.0,5.0,73.0,103.0,3.0,0.0,4.0,1.0,48301,MI,42.543278,-83.279277
6,2,Bloomfield Hills L2,6.0,20.0,8.0,28.0,21.0,26.0,5.0,87.0,139.0,3.0,0.0,4.0,1.0,48303,MI,42.584000,-83.243000
7,2,Bloomfield Hills L3,6.0,23.0,8.0,28.0,21.0,26.0,5.0,88.0,144.0,3.0,0.0,4.0,1.0,48302,MI,42.592675,-83.294449
8,2,Bloomfield Hills L4,6.0,23.0,8.0,28.0,21.0,26.0,5.0,88.0,145.0,3.0,0.0,4.0,1.0,48304,MI,42.588184,-83.234946
9,2,Bloomfield township,6.0,17.0,8.0,27.0,21.0,25.0,5.0,83.0,138.0,3.0,0.0,4.0,1.0,48301,MI,42.545294,-83.260391


#### 6. Using Folium Maps to represent clusters on a map

In [16]:
map_clusters = folium.Map(location = [42.655822,-83.229792], zoom_start=10)

folium.Marker([42.655822,-83.229792], popup='User Work Location - FCA Auburn Hills',icon=folium.Icon(color='red')).add_to(map_clusters)
folium.Circle([42.655822,-83.229792],radius=25749.5).add_to(map_clusters)

x = np.arange(k_cluster)
ys = [i + x + (i*x)**2 for i in range(k_cluster)]
colors_array = ['green', 'blue','orange', 'red', 'cadetblue']
for lat,lon,n,cluster in zip(df_cluster['Latitude'], df_cluster['Longitude'], df_cluster['Neighborhood'], df_cluster['Cluster Labels']):
    label = folium.Popup(str(n) + ' Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=4,
    popup=label,
    color=colors_array[cluster-1],
    fill=True,
    fill_color=colors_array[cluster-1],
    fill_opacity=1).add_to(map_clusters)

map_clusters

#### 7. Analyzing Clusters

In [17]:
df_cluster1 = df_cluster.loc[df_cluster['Cluster Labels']==0, df_cluster.columns[[1]+list(range(2,df_cluster.shape[1]))]]
df_cluster1.reset_index(drop=True, inplace=True)
av_cluster1 = df_cluster1.mean(axis=0).round(2)
df_cluster_avg_1 = df_cluster1.append(av_cluster1,ignore_index=True)
df_cluster_avg_1.head()

df_cluster2 = df_cluster.loc[df_cluster['Cluster Labels']==1, df_cluster.columns[[1]+list(range(2,df_cluster.shape[1]))]]
df_cluster2.reset_index(drop=True, inplace=True)
av_cluster2 = df_cluster2.mean(axis=0).round(2)
df_cluster_avg_2 = df_cluster2.append(av_cluster2,ignore_index=True)
df_cluster_avg_2.head()

df_cluster3 = df_cluster.loc[df_cluster['Cluster Labels']==2, df_cluster.columns[[1]+list(range(2,df_cluster.shape[1]))]]
df_cluster3.reset_index(drop=True, inplace=True)
av_cluster3 = df_cluster3.mean(axis=0).round(2)
df_cluster_avg_3 = df_cluster3.append(av_cluster3,ignore_index=True)
df_cluster_avg_3.head()

df_cluster4 = df_cluster.loc[df_cluster['Cluster Labels']==3, df_cluster.columns[[1]+list(range(2,df_cluster.shape[1]))]]
df_cluster4.reset_index(drop=True, inplace=True)
av_cluster4 = df_cluster4.mean(axis=0).round(2)
df_cluster_avg_4 = df_cluster4.append(av_cluster4,ignore_index=True)
df_cluster_avg_4.head()

df_cluster5= df_cluster.loc[df_cluster['Cluster Labels']==4, df_cluster.columns[[1]+list(range(2,df_cluster.shape[1]))]]
df_cluster5.reset_index(drop=True, inplace=True)
av_cluster5 = df_cluster4.mean(axis=0).round(2)
df_cluster_avg_5 = df_cluster4.append(av_cluster4,ignore_index=True)
df_cluster_avg_5.head()


,Neighborhood,Arts & Crafts Store,Convenience Store,Grocery Store,Hardware Store,Mediterranean Restaurant,Mexican Restaurant,Noodle House,Pharmacy,Pizza Place,Shipping Store,Supermarket,Taco Place,nan,Zip Code,State,Latitude,Longitude
0,Keego Harbor,6.0,33.0,11.0,36.0,24.0,32.0,5.0,99.0,179.0,4.0,1.0,7.0,1.0,48320.0,MI,42.607167,-83.339980
1,Pontiac L1,6.0,35.0,11.0,36.0,24.0,34.0,5.0,99.0,188.0,4.0,1.0,8.0,1.0,48341.0,MI,42.625769,-83.301903
2,Pontiac L2,7.0,38.0,11.0,36.0,26.0,39.0,5.0,99.0,196.0,4.0,1.0,8.0,1.0,48342.0,MI,42.656350,-83.286390
3,Pontiac L3,7.0,38.0,11.0,36.0,26.0,39.0,5.0,99.0,196.0,4.0,1.0,8.0,1.0,48343.0,MI,42.638100,-83.302400
4,Rochester L1,8.0,40.0,12.0,39.0,27.0,39.0,5.0,101.0,217.0,5.0,1.0,8.0,1.0,48309.0,MI,42.660261,-83.193320


#### 8. Making Realtor API query to get list of apartments in the chosen cluster

In [19]:
df_chosen_cluster = df_cluster3

filter_columns = ['Neighborhood','Zip Code']
df_cluster_org = df_chosen_cluster[filter_columns]

list_cat1 = []
list_cat2 = []

for neigh,zipcode in zip(df_cluster_org['Neighborhood'], df_cluster_org['Zip Code']):
    
    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-rent"
    querystring = {"sort":"relevance","radius":"0.01","postal_code":zipcode,"city":"Michigan","state_code":"MI","limit":"10","offset":"0"}
    headers = {
            'x-rapidapi-host': "realtor.p.rapidapi.com",
            'x-rapidapi-key': "6be9b25835mshf041f9e3beab781p19b7c3jsn77f325415059"
            }
    
    print(neigh)

    response = requests.request("GET", url, headers=headers, params=querystring)
    results = response.json()
    venues = results['properties']
    dataframe = json_normalize(venues)
    filtered_columns =[col for col in dataframe.columns if col.startswith('address.')] +[col for col in dataframe.columns if col.startswith('community.')] + [col for col in dataframe.columns if col.startswith('price')]
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    length = len(dataframe_filtered.columns)

    if length>15:
        col = ['line','postal_code','city','name', 'price_max', 'price_min', 'lat', 'lon']
        list_cat1.append(dataframe_filtered[col])
        
    else:
        col = ['line','postal_code','city','price','lat','lon']
        list_cat2.append(dataframe_filtered[col])
        
df_cat1 = pd.concat(list_cat1, axis=0)
 
df_cat2 = pd.concat(list_cat2, axis=0)
df_cat2['name'] = df_cat2['line']
df_cat2['price_max'] = df_cat2['price']
df_cat2['price_min'] = df_cat2['price']
 
df_apartments = pd.concat([df_cat1,df_cat2], axis=0, ignore_index=True)
df_apartments.drop(['price'], axis=1, inplace=True)
df_apartments.dropna(inplace=True)
df_apartments.reset_index(drop=True, inplace=True)
df_apartments.head(15)


Bloomfield Hills L2
Bloomfield Hills L3
Bloomfield Hills L4
Bloomfield township 
Commerce Township
Sterling Heights L1
Sterling Heights L2
Troy L2
Troy L3
Troy L4
Troy L5
West Bloomfield L2
West Bloomfield L3
West Bloomfield L4


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,city,lat,line,lon,name,postal_code,price_max,price_min
0,Bloomfield Hills,42.606620,1610-1681 Bloomfield Place Dr,-83.301640,Bloomfield Place Apartments,48302,979.0,849.0
1,West Bloomfield,42.602100,2510 Woodrow Wilson Blvd,-83.339430,Bloomfield on the Green Apartments,48324,1028.0,783.0
2,Pontiac,42.618893,850 Golf Dr,-83.305505,Crystal Lake Apartments,48341,999.0,899.0
3,Bloomfield Hills,42.554710,5940 Cohasset Ln,-83.283890,Whethersfield,48301,1675.0,1110.0
4,Pontiac,42.616164,900 Martin Luther King Jr Blvd S,-83.272590,Fox Pointe Apartments,48341,855.0,725.0
5,Pontiac,42.626270,311 S Telegraph Rd,-83.324730,Bloomfield On The River,48341,720.0,595.0
6,Auburn Hills,42.621020,3161 Bloomfield Ln,-83.227300,Bloomfield Square Apartments,48326,1075.0,955.0
7,Auburn Hills,42.636980,124 Optimist Rd,-83.210600,Parkways of Auburn Hills,48326,2075.0,1625.0
8,Birmingham,42.545450,1997 Villa Rd,-83.196330,Eton Square,48009,2000.0,975.0
9,Pontiac,42.616164,900 Martin Luther King Jr Blvd S,-83.272590,Fox Pointe Apartments,48341,855.0,725.0


#### 9. Shorlisting Apartments based on Customer Requirements

In [23]:
df_mod= df_apartments.drop_duplicates(subset=['name'], keep = 'first')
df_mod.reset_index(inplace=True, drop=True)
df_mod_new = df_mod.drop_duplicates(subset=['line'], keep = 'first')

user_minprice = 1100
user_maxprice = 2000

min_check =  df_mod_new['price_min'] >=user_minprice-50
max_check =  df_mod_new['price_max'] <=user_maxprice-50

df_sort = df_mod_new[min_check & max_check]
df_sort.reset_index(drop=True, inplace=True)
df_sort.sort_values(by='price_max', ascending = True, inplace=True)
df_sort.reset_index(inplace=True, drop=True)
df_sort.rename(columns={"city": "Neighborhood", "lat": "Latitude", "lon": "Longitude", "line": "Address", "name": "Apartment Name", "postal_code": "Zip Code", "price_max": "Max Price($)", "price_min": "Min Price($)"}, inplace=True)
df_sort.drop(df_sort.index[4], inplace=True)
df_sort.reset_index(inplace=True, drop=True)

for lat,lon,name,zipcode in zip(df_sort['Latitude'], df_sort['Longitude'],df_sort['Apartment Name'], df_sort['Zip Code']):
    label = folium.Popup(str(name) + ',' + str(zipcode), parse_html=True)
    folium.Marker(
    [lat,lon],
    popup=label,icon=folium.Icon(color='black')
    ).add_to(map_clusters)
    
body4 = client_8dbf0725bdee4042881dc53e79d63a02.get_object(Bucket='capstoneproject-donotdelete-pr-enuemg2yqmnfxz',Key='Outliers_Apartments.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body4, "__iter__"): body4.__iter__ = types.MethodType( __iter__, body4 )

df_outlier = pd.read_csv(body4)
    

for lat,lon,name,zipcode in zip(df_outlier['Latitude'], df_outlier['Longitude'],df_outlier['Apartment Name'], df_outlier['Zip Code']):
    label = folium.Popup(str(name) + ',' + str(zipcode), parse_html=True)
    folium.Marker(
    [lat,lon],
    popup=label,icon=folium.Icon(color='purple')
    ).add_to(map_clusters)

map_clusters

df_sort.drop(df_sort.index[0], inplace=True)
df_sort.reset_index(inplace=True, drop=True)

print ('****** List of Shorlisted Apartments ******')
df_sort.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


****** List of Shorlisted Apartments ******


,Neighborhood,Latitude,Address,Longitude,Apartment Name,Zip Code,Max Price($),Min Price($)
0,Clinton Township,42.556314,35700 Moravian Dr,-82.968635,Woodland Meadows Apts LLC,48035,1379.0,1071.0
1,Commerce Township,42.596130,2649 Grove Cir,-83.448250,Redwood Commerce Township,48382,1600.0,1600.0
2,Bloomfield Hills,42.585827,1733 Huntingwood Ln Apt C,-83.239238,1733 Huntingwood Ln Apt C,48304,1650.0,1650.0
3,Birmingham,42.539410,505 E Lincoln St,-83.210710,Lincoln House,48009,1790.0,1790.0
4,Bloomfield Hills,42.583719,150 E Long Lake Rd Apt 3,-83.240218,150 E Long Lake Rd Apt 3,48304,1800.0,1800.0
5,Bloomfield Hills,42.531870,7480 Bingham Rd,-83.283590,Glens of Bloomfield Apartments & Townhomes,48301,1825.0,1205.0
6,Royal Oak,42.527270,4301-4595 Coolidge Hwy,-83.184600,Amber Oak Townhomes,48073,1845.0,1825.0
7,Troy,42.556983,3100 Glouchester Ave,-83.188399,Buckingham Square Apts and TH,48084,1850.0,1050.0
